In [30]:
%pip install -U langchain langchain-openai

from langchain.prompts import PromptTemplate  # LangChain의 프롬프트 템플릿 모듈
from langchain_openai import ChatOpenAI  # OpenAI의 챗봇 모델을 사용하기 위한 모듈
from langchain.schema import StrOutputParser  # 모델의 출력을 문자열로 변환하는 모듈
from langchain.schema.runnable import RunnablePassthrough  # 데이터 변환 없이 그대로 전달하는 모듈
import os  # 환경 변수 설정을 위한 모듈

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from dotenv import load_dotenv

load_dotenv()  # .env 파일 로드
api_key = os.getenv("OPENAI_API_KEY")


In [32]:
# 여행 가이드 챗봇 프롬프트 템플릿 설정
tour_guide_template = """
당신은 여행을 좋아하는 10년차 여행 가이드입니다.
사용자가 입력한 여행 국가와 도시에 맞춰 3일간의 가상 여행 계획을 작성해주세요.
반드시 아래 FORMAT에 맞춰 작성하세요.

#상황:
{question}

#FORMAT:
- Day1: 
- Day2: 
- Day3: 
"""

In [33]:
# 사용자로부터 여행 국가와 도시를 입력받음
country = input("여행 국가를 입력하세요: ")  # 예: "프랑스"
city = input("여행 도시를 입력하세요: ")  # 예: "파리"

In [34]:
# 입력받은 국가와 도시를 하나의 문자열로 구성
# 입력값을 포맷에 맞춰 문자열로 변환
user_input = f"여행 국가: {country}, 여행 도시: {city}"  # "여행 국가: <국가>, 여행 도시: <도시>" 형식으로 만들어야 함

In [35]:
# LangChain의 프롬프트 템플릿을 적용하여 사용자 입력을 프롬프트에 반영
prompt = PromptTemplate.from_template(user_input)  # 프롬프트 템플릿을 이용하여 객체 생성

In [ ]:
# OpenAI의 GPT-4o-mini 모델을 활용하여 여행 일정 생성
model = ChatOpenAI(model_name="gpt-5-nano")  # OpenAI의 GPT-4o-mini 모델 사용 (gpt-5-nano 모델 사용 가능한 api key 였음)

In [37]:
# 모델의 출력을 문자열로 변환하는 파서 (결과 데이터를 가공 없이 그대로 출력할 수 있도록 설정)
output_parser = StrOutputParser()  # 모델 결과를 문자열로 변환하는 파서

In [47]:
# 체인 구성: 
# 1. 사용자 입력값을 그대로 전달 (RunnablePassthrough)
# 2. 프롬프트 템플릿 적용
# 3. OpenAI 모델을 실행하여 답변 생성
# 4. 생성된 답변을 문자열로 변환하여 반환
# 체인 구성: 사용자 입력 → 변환(RunnablePassthrough) → 프롬프트 적용 → 모델 실행 → 출력 변환
chain = { "question": RunnablePassthrough() } | prompt | model | output_parser

In [48]:
# 체인을 실행하여 여행 계획을 생성
# 사용자가 입력한 국가와 도시 정보를 `question` 키에 매핑하여 전달
answer = chain.invoke({"question": user_input})  # 체인을 실행하고 결과를 가져옴

In [49]:
# 생성된 여행 계획 출력
print("\n[여행 계획]")  # 여행 계획 제목 출력
print(answer)  # 모델이 생성한 여행 계획 출력


[여행 계획]
좋아요! 프랑스 파리 여행을 잘 준비하시도록 도와드릴게요. 원하시는 일정, 예산, 관심사, 방문 날짜를 알려주시면 더 맞춤형으로 만들어 드리겠습니다. 우선 기본 가이드와 간단한 샘플 코스를 드려요.

기본 팁
- 핵심 명소: 루브르 박물관, 에펠탑, 몽마르트르의 사크레쾨르 대성당, 오르세 미술관, 센강 크루즈, 노트르담 주변(외관 관람 가능), 샤를리 드 모나코? (정확히는 생제르맹-데-쁘) 등.
- 동선 팁: 파리는 도보 + 지하철이 편리합니다. 가능하면 하루에 1~2곳으로 집중하고, 미리 티켓 온라인 예매를 권장합니다(대부분의 인기 미술관은 시간대 입장권이 필요).
- 교통: 지하철/버스/RER 이용. 관광객용 패스도 있지만 일정에 맞춰 티켓을 구입하는 편이 낫습니다. 카드 결제 널리 사용.
- 식사 팁: 빵집(바게트, 크루아상), 카페에서의 커피와 크루아상, 크레페, 샤퀴테리와 치즈, 와인도 즐기기 좋습니다.
- 안전: 피크 시간대 관광지에서 소지품 분실 주의. 좁은 골목이나 인적이 드문 곳은 야간에는 주의.
- 계절: 봄/가을은 방문하기 좋고, 여름은 야외 활동과 인파가 많습니다. 겨울은 크리스마스 기간 빼고 한산한 편.

샘플 코스
- 3일 클래식 코스
  - 1일차: 루브르 박물관(오전), 튈르리 정원 산책, 오르세 미술관 방문, 센 강 야경 크루즈(저녁)
  - 2일차: Île de la Cité의 Sainte-Chapelle과 Conciergerie 방문, 노트르담 외관 감상(대부분 복구로 내부 관람은 제한될 수 있음), 몽마르트르 언덕과 사크레쾨르 대성당 방문, 몽마르트르 골목 산책
  - 3일차: 에펠탑 조망 후 샤를 드 골샤넬 거리(샹젤리제)와 개선문 방문, 센강가 산책
- 4–5일 여유 코스
  - 1일차: 루브르 + 튈르리 정원 + 세느 강 크루즈
  - 2일차: Île Saint-Louis 포함 센느 강 마을 분위기, 마레 지역 산책(콘시엘유지)
  - 3일차: 몽마르트르 및 피나토몰(예술적 분위기) 방문
  - 4일차